#### Importing Libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import csv

#### Importing Data

In [ ]:
raw_data = open('data.csv', 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
data = list(reader)
data = np.array(data)
data = data[1:, 2:].astype(float)

#### Data Processing 
- Sequences are made into a square image of closest binary number to the the ceiling of the root of the length

In [ ]:
rows, columns = np.shape(data)
sqrt_col = int(np.ceil(np.sqrt(columns)))
processed_data = np.zeros([rows, 256, 256])
data_ = np.zeros([rows, 256*256])

for i in range(rows):
    data[i] = (data[i] - np.mean(data[i]))/np.max(data[i])
    data_[i] = np.append(data[i], np.zeros(256*256 - columns))

    
for i in range(rows):
    for j in range(sqrt_col):
            processed_data[i,j,:] = data_[i, j * 256 : (j+1)*256]

#### Generator

In [ ]:
def gen_model(input_shape):
    
    Input_1= tf.keras.layers.Input(shape = input_shape)
    
    model = tf.keras.layers.Conv2D(256, (10,10), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(Input_1)
    model = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.layers.Conv2D(128, (10,10), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.layers.Conv2D(128, (7,7), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.layers.Conv2D(128, (5,5), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.layers.Conv2D(1, (5,5), activation='tanh', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    output = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.models.Model(inputs=Input_1, outputs = output, name = 'generator')
    
    return model

In [ ]:
gen = gen_model((256,256,1))

In [ ]:
gen.summary()

#### Discriminator


In [ ]:
def discrim_model(input_shape):
    
    Input_1= tf.keras.layers.Input(shape = input_shape)
    
    model = tf.keras.layers.Conv2D(256, (10,10), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(Input_1)
    model = tf.keras.layers.BatchNormalization()(model)
    model = tf.keras.layers.MaxPool2D()(model)
    
    model = tf.keras.layers.Conv2D(128, (10,10), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    model = tf.keras.layers.MaxPool2D()(model)
    
    model = tf.keras.layers.Conv2D(128, (7,7), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    model = tf.keras.layers.MaxPool2D()(model)
    
    model = tf.keras.layers.Conv2D(128, (5,5), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    model = tf.keras.layers.MaxPool2D()(model)
    
    model = tf.keras.layers.Conv2D(64, (3,3), activation='tanh', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    model = tf.keras.layers.MaxPool2D()(model)
    model = tf.keras.layers.Flatten()(model)
    
    output = tf.keras.layers.Dense(1, activation = 'sigmoid')(model)
    
    model = tf.keras.models.Model(inputs=Input_1, outputs = output, name = 'discriminator')
    
    return model

In [ ]:
discrim = discrim_model((256,256,1))

In [ ]:
discrim.summary()

#### Creating the Entire Network

In [ ]:
# Discriminator
optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
discriminator = discrim_model((256, 256, 1))
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
# Generator
generator = gen_model((256,256,1))

In [ ]:
# Combined Network
inp = tf.keras.Input((256, 256,1))
gen = generator(inp)
discriminator.trainable = False
discrim = discriminator(gen)
gan = tf.keras.Model(inp, discrim)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
# Summary
gan.summary()

#### Training

In [ ]:
# Training Parameters
batch_size = 50
iterations = 100

# Training Data
X_train = np.expand_dims(processed_data, axis = 3)

# Labels
true_label = np.ones((batch_size, 1))
fake_label = np.zeros((batch_size, 1))

# Training
for epoch in range(iterations):
    
    # Random sample of data
    random_indices = np.random.randint(0, X_train.shape[0], batch_size)
    discrim_data = X_train[random_indices]
    
    # Training the discriminator
    gaussian = np.random.normal(0, 1, (batch_size, X_train.shape[1], X_train.shape[2], 1))
    discrim_loss_real = discriminator.train_on_batch(discrim_data, true_label)
    generated_data = generator.predict(gaussian)
    discriminator_loss_real = self.discriminator.train_on_batch(discrim_data, true_label)
    discriminator_loss_fake = self.discriminator.train_on_batch(generated_data, fake_label)
    discriminator_loss_total = 0.5 * np.add(discrim_loss_real, discriminator_loss_fake)
    
    # Training the gan
    gan.train_on_batch(gaussian, true_label)
    
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))